In [ ]:
# Install required packages
%pip install transformers torch torchvision pillow opencv-python matplotlib numpy gradio plotly


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
from typing import Tuple, Dict, List
import time
import json

# Transformers and models
from transformers import pipeline, DPTImageProcessor, DPTForDepthEstimation
from transformers import AutoImageProcessor, AutoModelForDepthEstimation

# Visualization
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Create directories
import os
os.makedirs("drone_test_images", exist_ok=True)
os.makedirs("obstacle_analysis", exist_ok=True)
os.makedirs("calibration_data", exist_ok=True)

warnings.filterwarnings('ignore')
print("✅ All dependencies imported successfully!")


In [ ]:
class DroneDepthEstimator:
    """
    A class to handle multiple depth estimation models for drone obstacle avoidance
    """
    
    def __init__(self):
        self.models = {}
        self.current_model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🔧 Using device: {self.device}")
        
    def load_model(self, model_name: str, model_id: str):
        """Load a depth estimation model"""
        try:
            print(f"📥 Loading {model_name}...")
            
            if "dpt" in model_id.lower():
                # DPT models
                processor = DPTImageProcessor.from_pretrained(model_id)
                model = DPTForDepthEstimation.from_pretrained(model_id)
            else:
                # Other models
                processor = AutoImageProcessor.from_pretrained(model_id)
                model = AutoModelForDepthEstimation.from_pretrained(model_id)
            
            model.to(self.device)
            model.eval()
            
            self.models[model_name] = {
                'processor': processor,
                'model': model,
                'model_id': model_id
            }
            
            print(f"✅ {model_name} loaded successfully!")
            return True
            
        except Exception as e:
            print(f"❌ Failed to load {model_name}: {str(e)}")
            return False
    
    def set_current_model(self, model_name: str):
        """Set the current active model"""
        if model_name in self.models:
            self.current_model = model_name
            print(f"🎯 Current model set to: {model_name}")
        else:
            print(f"❌ Model {model_name} not found!")
    
    def list_available_models(self):
        """List all loaded models"""
        print("📋 Available models:")
        for name, info in self.models.items():
            status = "🎯 ACTIVE" if name == self.current_model else "⚪ INACTIVE"
            print(f"  - {name} ({info['model_id']}) {status}")

# Initialize the depth estimator
depth_estimator = DroneDepthEstimator()


In [ ]:
# Load multiple depth estimation models for comparison
models_to_load = [
    ("DPT-Large", "Intel/dpt-large"),
    ("DPT-Hybrid", "Intel/dpt-hybrid-midas"),
    ("MiDaS", "Intel/midas-v2-1-small-256"),
]

print("🚀 Loading depth estimation models for drone navigation...\n")

for model_name, model_id in models_to_load:
    success = depth_estimator.load_model(model_name, model_id)
    if success and depth_estimator.current_model is None:
        depth_estimator.set_current_model(model_name)
    print()

print("🎉 Model loading complete!")
depth_estimator.list_available_models()


In [ ]:
def predict_depth(image: np.ndarray, model_name: str = None) -> Tuple[np.ndarray, Dict]:
    """
    Predict depth map from image using specified model
    
    Args:
        image: Input RGB image (numpy array)
        model_name: Name of model to use (if None, uses current model)
        
    Returns:
        depth_map: Depth map as numpy array
        metadata: Processing metadata and statistics
    """
    if model_name and model_name in depth_estimator.models:
        model_info = depth_estimator.models[model_name]
    elif depth_estimator.current_model:
        model_info = depth_estimator.models[depth_estimator.current_model]
        model_name = depth_estimator.current_model
    else:
        raise ValueError("No model available for depth prediction!")
    
    start_time = time.time()
    
    # Convert numpy array to PIL Image
    if isinstance(image, np.ndarray):
        if image.shape[2] == 3:  # RGB
            pil_image = Image.fromarray(image)
        else:
            pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        pil_image = image
    
    # Process image
    processor = model_info['processor']
    model = model_info['model']
    
    inputs = processor(images=pil_image, return_tensors="pt").to(depth_estimator.device)
    
    # Predict depth
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth
    
    # Convert to numpy and normalize
    depth_map = predicted_depth.squeeze().cpu().numpy()
    
    processing_time = time.time() - start_time
    
    metadata = {
        'model_used': model_name,
        'processing_time': processing_time,
        'min_depth': float(depth_map.min()),
        'max_depth': float(depth_map.max()),
        'mean_depth': float(depth_map.mean()),
        'image_shape': image.shape,
        'depth_shape': depth_map.shape
    }
    
    return depth_map, metadata


In [ ]:
def calibrate_depth_to_distance(depth_map: np.ndarray, 
                               min_distance: float = 0.5, 
                               max_distance: float = 10.0) -> np.ndarray:
    """
    Calibrate relative depth values to real-world distances (meters)
    
    Args:
        depth_map: Raw depth map from model
        min_distance: Minimum real-world distance in meters
        max_distance: Maximum real-world distance in meters
        
    Returns:
        distance_map: Calibrated distance map in meters
    """
    # Invert depth if needed (some models output inverse depth)
    if depth_map.min() < 0:
        depth_map = -depth_map
    
    # Normalize to 0-1 range
    depth_normalized = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
    
    # Map to distance range (inverse relationship: closer objects = smaller depth values)
    # For many models, smaller values represent closer objects
    distance_map = min_distance + (1 - depth_normalized) * (max_distance - min_distance)
    
    return distance_map


In [ ]:
class DroneNavigationAnalyzer:
    """
    Analyze depth maps for drone navigation and obstacle avoidance
    """
    
    def __init__(self, 
                 danger_distance: float = 1.5,
                 warning_distance: float = 3.0,
                 safe_distance: float = 5.0):
        """
        Initialize navigation analyzer with safety thresholds
        
        Args:
            danger_distance: Distance threshold for danger zone (meters)
            warning_distance: Distance threshold for warning zone (meters)
            safe_distance: Distance threshold for safe zone (meters)
        """
        self.danger_distance = danger_distance
        self.warning_distance = warning_distance
        self.safe_distance = safe_distance
        
    def create_safety_zones(self, distance_map: np.ndarray) -> Tuple[np.ndarray, Dict]:
        """
        Create color-coded safety zones for navigation
        
        Args:
            distance_map: Distance map in meters
            
        Returns:
            safety_visualization: RGB image with color-coded zones
            zone_statistics: Statistics for each zone
        """
        h, w = distance_map.shape
        safety_zones = np.zeros((h, w, 3), dtype=np.uint8)
        
        # Define zone masks
        danger_mask = distance_map < self.danger_distance
        warning_mask = (distance_map >= self.danger_distance) & (distance_map < self.warning_distance)
        caution_mask = (distance_map >= self.warning_distance) & (distance_map < self.safe_distance)
        safe_mask = distance_map >= self.safe_distance
        
        # Color-code zones (BGR format)
        safety_zones[danger_mask] = [0, 0, 255]      # Red - DANGER
        safety_zones[warning_mask] = [0, 165, 255]   # Orange - WARNING
        safety_zones[caution_mask] = [0, 255, 255]   # Yellow - CAUTION
        safety_zones[safe_mask] = [0, 255, 0]        # Green - SAFE
        
        # Calculate statistics
        total_pixels = h * w
        zone_stats = {
            'danger_pixels': np.sum(danger_mask),
            'warning_pixels': np.sum(warning_mask),
            'caution_pixels': np.sum(caution_mask),
            'safe_pixels': np.sum(safe_mask),
            'danger_percentage': (np.sum(danger_mask) / total_pixels) * 100,
            'warning_percentage': (np.sum(warning_mask) / total_pixels) * 100,
            'caution_percentage': (np.sum(caution_mask) / total_pixels) * 100,
            'safe_percentage': (np.sum(safe_mask) / total_pixels) * 100,
            'closest_obstacle': float(distance_map.min()),
            'average_distance': float(distance_map.mean())
        }
        
        return safety_zones, zone_stats
        
    def analyze_flight_path(self, distance_map: np.ndarray) -> Dict:
        """
        Analyze the image for safe flight directions
        
        Args:
            distance_map: Distance map in meters
            
        Returns:
            flight_analysis: Recommended flight directions and safety assessment
        """
        h, w = distance_map.shape
        
        # Divide image into sectors for directional analysis
        center_h, center_w = h // 2, w // 2
        
        # Define sectors
        sectors = {
            'forward': distance_map[center_h-h//4:center_h+h//4, center_w-w//4:center_w+w//4],
            'left': distance_map[:, :w//3],
            'right': distance_map[:, 2*w//3:],
            'up': distance_map[:h//3, :],
            'down': distance_map[2*h//3:, :],
            'forward_left': distance_map[center_h-h//4:center_h+h//4, :w//2],
            'forward_right': distance_map[center_h-h//4:center_h+h//4, w//2:]
        }
        
        # Analyze each sector
        analysis = {}
        for direction, sector in sectors.items():
            min_dist = float(sector.min())
            mean_dist = float(sector.mean())
            
            # Determine safety level
            if min_dist < self.danger_distance:
                safety_level = "DANGER"
                recommendation = "AVOID"
            elif min_dist < self.warning_distance:
                safety_level = "WARNING"
                recommendation = "CAUTION"
            elif min_dist < self.safe_distance:
                safety_level = "CAUTION"
                recommendation = "PROCEED_CAREFULLY"
            else:
                safety_level = "SAFE"
                recommendation = "CLEAR"
            
            analysis[direction] = {
                'min_distance': min_dist,
                'mean_distance': mean_dist,
                'safety_level': safety_level,
                'recommendation': recommendation,
                'safe_percentage': (np.sum(sector >= self.safe_distance) / sector.size) * 100
            }
        
        # Overall flight recommendation
        forward_safety = analysis['forward']['safety_level']
        closest_obstacle = float(distance_map.min())
        
        if closest_obstacle < self.danger_distance:
            overall_recommendation = "EMERGENCY_STOP"
        elif forward_safety == "SAFE":
            overall_recommendation = "CONTINUE_FORWARD"
        elif analysis['left']['safety_level'] == "SAFE":
            overall_recommendation = "TURN_LEFT"
        elif analysis['right']['safety_level'] == "SAFE":
            overall_recommendation = "TURN_RIGHT"
        elif analysis['up']['safety_level'] == "SAFE":
            overall_recommendation = "CLIMB"
        else:
            overall_recommendation = "HOVER_AND_REASSESS"
        
        analysis['overall'] = {
            'recommendation': overall_recommendation,
            'closest_obstacle': closest_obstacle,
            'confidence': self._calculate_confidence(analysis)
        }
        
        return analysis
    
    def _calculate_confidence(self, analysis: Dict) -> float:
        """Calculate confidence score for navigation recommendations"""
        # Simple confidence calculation based on distance margins
        min_distances = [data['min_distance'] for data in analysis.values() if isinstance(data, dict)]
        if not min_distances:
            return 0.5
        
        avg_min_distance = np.mean(min_distances)
        
        if avg_min_distance >= self.safe_distance:
            return 0.9
        elif avg_min_distance >= self.warning_distance:
            return 0.7
        elif avg_min_distance >= self.danger_distance:
            return 0.4
        else:
            return 0.2

# Initialize navigation analyzer
nav_analyzer = DroneNavigationAnalyzer()
print("🧭 Navigation analyzer initialized with default safety thresholds")


In [ ]:
def create_comprehensive_visualization(image: np.ndarray, 
                                     distance_map: np.ndarray, 
                                     safety_zones: np.ndarray,
                                     flight_analysis: Dict,
                                     zone_stats: Dict) -> np.ndarray:
    """
    Create a comprehensive visualization for drone navigation
    
    Args:
        image: Original RGB image
        distance_map: Distance map in meters
        safety_zones: Color-coded safety zones
        flight_analysis: Flight path analysis results
        zone_stats: Zone statistics
        
    Returns:
        visualization: Combined visualization image
    """
    h, w = image.shape[:2]
    
    # Create a larger canvas for multiple views
    canvas_height = h * 2 + 100  # Extra space for text
    canvas_width = w * 2 + 100
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
    
    # 1. Original image (top-left)
    canvas[50:50+h, 50:50+w] = image
    
    # 2. Distance heatmap (top-right)
    distance_normalized = cv2.normalize(distance_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    distance_colored = cv2.applyColorMap(distance_normalized, cv2.COLORMAP_VIRIDIS)
    canvas[50:50+h, 50+w+50:50+w+50+w] = distance_colored
    
    # 3. Safety zones (bottom-left)
    canvas[50+h+50:50+h+50+h, 50:50+w] = safety_zones
    
    # 4. Combined overlay (bottom-right)
    overlay = cv2.addWeighted(image, 0.6, safety_zones, 0.4, 0)
    canvas[50+h+50:50+h+50+h, 50+w+50:50+w+50+w] = overlay
    
    # Add labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    color = (255, 255, 255)
    thickness = 2
    
    cv2.putText(canvas, "Original Image", (50, 30), font, font_scale, color, thickness)
    cv2.putText(canvas, "Distance Map", (50+w+50, 30), font, font_scale, color, thickness)
    cv2.putText(canvas, "Safety Zones", (50, 50+h+30), font, font_scale, color, thickness)
    cv2.putText(canvas, "Navigation Overlay", (50+w+50, 50+h+30), font, font_scale, color, thickness)
    
    # Add navigation recommendation
    recommendation = flight_analysis['overall']['recommendation']
    confidence = flight_analysis['overall']['confidence']
    closest_dist = flight_analysis['overall']['closest_obstacle']
    
    # Color-code recommendation
    if recommendation == "EMERGENCY_STOP":
        rec_color = (0, 0, 255)  # Red
    elif recommendation in ["TURN_LEFT", "TURN_RIGHT", "CLIMB"]:
        rec_color = (0, 165, 255)  # Orange
    elif recommendation == "CONTINUE_FORWARD":
        rec_color = (0, 255, 0)  # Green
    else:
        rec_color = (0, 255, 255)  # Yellow
    
    # Add recommendation text
    rec_text = f"RECOMMENDATION: {recommendation.replace('_', ' ')}"
    cv2.putText(canvas, rec_text, (50, canvas_height - 60), font, 0.7, rec_color, 2)
    
    conf_text = f"Confidence: {confidence:.1%} | Closest: {closest_dist:.1f}m"
    cv2.putText(canvas, conf_text, (50, canvas_height - 30), font, 0.6, (255, 255, 255), 2)
    
    return canvas


In [ ]:
def create_directional_analysis_plot(flight_analysis: Dict):
    """
    Create a plotly radar chart showing directional safety analysis
    """
    directions = ['forward', 'left', 'right', 'up', 'down', 'forward_left', 'forward_right']
    
    # Extract data for radar chart
    min_distances = [flight_analysis[d]['min_distance'] for d in directions]
    safety_percentages = [flight_analysis[d]['safe_percentage'] for d in directions]
    
    # Create radar chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=min_distances,
        theta=directions,
        fill='toself',
        name='Min Distance (m)',
        line_color='blue'
    ))
    
    fig.add_trace(go.Scatterpolar(
        r=[d/20 for d in safety_percentages],  # Normalize to similar scale
        theta=directions,
        fill='toself',
        name='Safety % (scaled)',
        line_color='green',
        opacity=0.6
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max(min_distances) * 1.1]
            )),
        showlegend=True,
        title="🧭 Directional Safety Analysis"
    )
    
    return fig


In [ ]:
def process_drone_navigation(image: np.ndarray, 
                           model_name: str = None,
                           min_distance: float = 0.5,
                           max_distance: float = 10.0,
                           danger_threshold: float = 1.5,
                           warning_threshold: float = 3.0,
                           safe_threshold: float = 5.0) -> Dict:
    """
    Complete pipeline for drone navigation analysis
    
    Args:
        image: Input RGB image
        model_name: Depth estimation model to use
        min_distance: Minimum calibration distance (meters)
        max_distance: Maximum calibration distance (meters)
        danger_threshold: Danger zone threshold (meters)
        warning_threshold: Warning zone threshold (meters)
        safe_threshold: Safe zone threshold (meters)
        
    Returns:
        results: Complete analysis results with visualizations
    """
    start_time = time.time()
    
    # Step 1: Predict depth
    print("🔍 Predicting depth...")
    depth_map, depth_metadata = predict_depth(image, model_name)
    
    # Step 2: Calibrate to real-world distances
    print("📏 Calibrating distances...")
    distance_map = calibrate_depth_to_distance(depth_map, min_distance, max_distance)
    
    # Step 3: Update navigation analyzer thresholds
    nav_analyzer.danger_distance = danger_threshold
    nav_analyzer.warning_distance = warning_threshold
    nav_analyzer.safe_distance = safe_threshold
    
    # Step 4: Create safety zones
    print("🚨 Analyzing safety zones...")
    safety_zones, zone_stats = nav_analyzer.create_safety_zones(distance_map)
    
    # Step 5: Analyze flight paths
    print("🧭 Analyzing flight paths...")
    flight_analysis = nav_analyzer.analyze_flight_path(distance_map)
    
    # Step 6: Create visualizations
    print("🎨 Creating visualizations...")
    comprehensive_viz = create_comprehensive_visualization(
        image, distance_map, safety_zones, flight_analysis, zone_stats
    )
    
    # Create directional analysis plot
    directional_plot = create_directional_analysis_plot(flight_analysis)
    
    total_time = time.time() - start_time
    
    # Compile results
    results = {
        'processing_time': total_time,
        'depth_metadata': depth_metadata,
        'distance_map': distance_map,
        'safety_zones': safety_zones,
        'zone_statistics': zone_stats,
        'flight_analysis': flight_analysis,
        'comprehensive_visualization': comprehensive_viz,
        'directional_plot': directional_plot,
        'calibration_params': {
            'min_distance': min_distance,
            'max_distance': max_distance,
            'danger_threshold': danger_threshold,
            'warning_threshold': warning_threshold,
            'safe_threshold': safe_threshold
        }
    }
    
    print(f"✅ Analysis complete in {total_time:.2f} seconds")
    return results


In [ ]:
# Download test images for different drone scenarios
test_images = [
    # Outdoor scenarios
    ("forest_path", "https://images.unsplash.com/photo-1441974231531-c6227db76b6e"),
    ("urban_street", "https://images.unsplash.com/photo-1506905925346-21bda4d32df4"),
    ("mountain_view", "https://images.unsplash.com/photo-1464822759844-d150baef493e"),
    
    # Indoor scenarios  
    ("corridor", "https://images.unsplash.com/photo-1586023492125-27b2c045efd7"),
    ("warehouse", "https://images.unsplash.com/photo-1553062407-98eeb64c6a62"),
    
    # Complex scenarios
    ("cluttered_room", "https://images.unsplash.com/photo-1555041469-a586c61ea9bc"),
]

print("📥 Downloading test images...")
import urllib.request

for name, url in test_images:
    try:
        filename = f"drone_test_images/{name}.jpg"
        urllib.request.urlretrieve(url, filename)
        print(f"✅ Downloaded: {name}")
    except Exception as e:
        print(f"❌ Failed to download {name}: {str(e)}")

print("\n🎉 Test images ready for drone navigation analysis!")
